Python example illustrating HTTP interactions with HSPC SMART Sandbox ...  
Log in, acquire bearer token, get userinfo, and query FHIR server.

In [ ]:
import getpass
import requests
import urllib
import uuid

define username and password

In [ ]:
username = 'jschneid@nmdp.org'
password = getpass.getpass()

other configuration settings

In [ ]:
hostname_prefix = 'cibmtr-smart-dev'
auth_server = 'https://{0}-auth.b12x.org'.format(hostname_prefix)
aud = 'https://{0}-stu3.b12x.org/smartstu3/data'.format(hostname_prefix)
sandman_url = 'https://{0}-sandman.b12x.org/'.format(hostname_prefix)
security_check_login_url = '{0}/j_spring_security_check'.format(auth_server)
client_id = 'sand_man'
response_type = 'code'
scope = 'smart/orchestrate_launch user/*.* profile openid'
submit = 'Sign in'

uncomment the following to authenticate as web-agnis-fhir-submission app (instead of sandbox manager) 

In [ ]:
#sandman_url = 'http://agnissubmission.b12x.org/'
#client_id = 'web-agnis-fhir-submission'

generate random state value for this login request

In [ ]:
state = str(uuid.uuid4())

See also:  
https://requests.readthedocs.io

get CapabilityStatement  
should include "http://fhir-registry.smarthealthit.org/StructureDefinition/oauth-uris" security extension, containing "authorize", "token", "register", and "launch-registration" URLs  
(currently ignored by this code)

In [ ]:
aud_metadata_url = '{0}/metadata'.format(aud)
r = requests.get(aud_metadata_url)

In [ ]:
r.status_code   # should be 200 (success)

response should be (JSON) CapabilityStatement containing security extensions for oauth-uris

In [ ]:
r.json()

TODO:  parse CapabilityStatement to obtain "authorize" and "token" URIs programmatically

construct URL for "authorize" endpoint

In [ ]:
authorize_url_fmt = '{0}/authorize?client_id={1}&response_type={2}&scope={3}&redirect_uri={4}&state={5}&aud={6}'
authorize_url = authorize_url_fmt.format(auth_server, urllib.parse.quote(client_id, safe=''), urllib.parse.quote(response_type, safe=''), urllib.parse.quote(scope, safe=''), urllib.parse.quote(sandman_url, safe=''), urllib.parse.quote(state, safe=''), urllib.parse.quote(aud, safe=''))

In [ ]:
authorize_url

send "authorize" request -- not yet logged in, so this should redirect to /login page

In [ ]:
r = requests.get(authorize_url, allow_redirects=False)

In [ ]:
r.status_code   # should be 302 (redirect)

In [ ]:
r.headers['set-cookie']

retain JSESSIONID cookie value, for use in subsequent request

In [ ]:
jsessionid = r.cookies.get('JSESSIONID')   # get JSESSIONID cookie
jsessionid

redirect location should be auth server's /login page

In [ ]:
r.headers['location']

request /login page

In [ ]:
r = requests.get(r.headers['location'], cookies=dict(JSESSIONID=jsessionid))

In [ ]:
r.status_code   # should be 200 (success)

In [ ]:
r.url

redisplay authorize_url

In [ ]:
authorize_url

log in with username and password -- successful login will be redirected to specified redirectUrl (authorize_url)

In [ ]:
r = requests.post(
    security_check_login_url,
    data = {'username': username, 'password': password, 'redirectUrl': authorize_url, 'submit': submit},
    cookies = dict(JSESSIONID=jsessionid),
    allow_redirects=False)

In [ ]:
r.status_code   # should be 302 (redirect)

redirect location should be same as authorize_url (the login request's redirectUrl)

In [ ]:
r.headers['location']

In [ ]:
authorize_url == r.headers['location']   # should be True

In [ ]:
r.headers['set-cookie']   # new JSESSIONID cookie value (?)

retain new JSESSIONID cookie value, for use in subsequent request

In [ ]:
jsessionid2 = r.cookies.get('JSESSIONID')
jsessionid2

In [ ]:
jsessionid == jsessionid2   # should be False

request authorization code -- using jsessionid2 now

In [ ]:
r = requests.get(
    r.headers['location'],
    cookies=dict(JSESSIONID=jsessionid2),
    allow_redirects=False)

In [ ]:
r.status_code   # should be 302 (redirect)

redirect location should be sandman home page (the previous request's redirect_uri), with code parameter added (the authorization code)

In [ ]:
r.headers['location']

parse redirect URL to get authorization code

In [ ]:
parsed_url = urllib.parse.urlsplit(r.headers['location'])
parsed_url

In [ ]:
parsed_url[3]

In [ ]:
code = urllib.parse.parse_qs(parsed_url[3])['code'][0]
code

use authorization code to get bearer token

In [ ]:
token_url = '{}/token'.format(auth_server)
r = requests.post(
    token_url,
    data = {'code': code, 'grant_type': 'authorization_code', 'redirect_uri': sandman_url, 'client_id': client_id},
    allow_redirects=False)

In [ ]:
r.status_code   # should be 200 (success)

response should be JSON containing bearer token (access_token)

In [ ]:
r.json()

In [ ]:
token_json = r.json()

granted token_json['scope'] should be equivalent to requested scope

In [ ]:
token_json['scope']

In [ ]:
scope

In [ ]:
'Bearer' == token_json['token_type']   # should be True

assemble value to be used in "authorization" header of subsequent requests

In [ ]:
bearer_token = 'BEARER ' + token_json['access_token']
bearer_token

prepare to request CapabilityStatement -- try OPTIONS request before sending GET request

In [ ]:
r = requests.options(aud_metadata_url)

In [ ]:
r.status_code   # should be 200 (success)

request CapabilityStatement

In [ ]:
r = requests.get(aud_metadata_url)

In [ ]:
r.status_code   # should be 200 (success)

response should be (JSON) CapabilityStatement

In [ ]:
r.json()

construct /userinfo request URL, then try OPTIONS request before sending POST request

In [ ]:
userinfo_url = '{}/userinfo'.format(auth_server)
r = requests.options(userinfo_url)

In [ ]:
r.status_code   # should be 200 (success)

use bearer token to request /userinfo

In [ ]:
r = requests.post(userinfo_url, headers={'authorization': bearer_token})

In [ ]:
r.status_code   # should be 200 (success)

response should be JSON containing info about user associated with bearer token

In [ ]:
r.json()

construct URL for FHIR Patient query

In [ ]:
patient_query_url = '{}/Patient?_sort:asc=family&_sort:asc=given&name=&_count=50'.format(aud)
patient_query_url

try OPTIONS request before sending GET request

In [ ]:
r = requests.options(patient_query_url)

In [ ]:
r.status_code   # should be 200 (success)

use bearer token to execute Patient query

In [ ]:
r = requests.get(patient_query_url, headers={'authorization': bearer_token})

In [ ]:
r.status_code   # should be 200 (success)

response should be (JSON) FHIR Bundle containing Patient entries

In [ ]:
r.json()

logout, with redirect back to home page

In [ ]:
logout_url = '{0}/logout?hspcRedirectUrl={1}'.format(auth_server, sandman_url)
r = requests.get(logout_url, cookies=dict(JSESSIONID=jsessionid2))

In [ ]:
r.status_code   # should be 200 (success)

In [ ]:
r.history   # should contain one <Response [302]>

In [ ]:
r.url   # should be sandman server's home page

-----
**STOP HERE** (for now)  

TODO -- replicate the above using fhirclient ?? ...

See also:  
https://smart-on-fhir.github.io/client-py  
https://github.com/smart-on-fhir/client-py

In [ ]:
from fhirclient import client

In [ ]:
settings = {
    'app_id': client_id,
    #'app_secret': '',
    'api_base': aud,
    'redirect_uri': sandman_url,
    #'patient_id': '',
    'scope': token_json['scope'],
    #'launch_token': ''
}
smart = client.FHIRClient(settings=settings)

In [ ]:
smart.ready

In [ ]:
smart.authorize_url

In [ ]:
smart.prepare()

In [ ]:
smart.authorize_url # is None

In [ ]:
import fhirclient.models.patient as p

In [ ]:
patient = p.Patient.read('2341', smart.server)